In [791]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from urllib.parse import quote
from tqdm import tqdm
import requests
import math
from selenium.webdriver.common.keys import Keys
import pandas as pd 

In [792]:
base_url = f'https://www.a-ha.io'
main_url = f'https://www.a-ha.io/search/%EB%B6%80%EB%8F%99%EC%82%B0%20%ED%8C%81'
driver = webdriver.Chrome()
driver.get(main_url)


In [793]:
# 파업창 닫기
driver.find_element(By.XPATH, '//*[@id="__layout"]/div/div[1]/div[10]/div[1]/div/div/div/div[4]/div[2]/button').click()

In [794]:
max_try = 30
for _ in range(max_try):
    more_btn = driver.find_element(By.CSS_SELECTOR, '.more10').click()
    time.sleep(1)

In [795]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [796]:
divs = soup.select('.box.tw-bg-transparent.min-height > div')
len(divs)

311

In [797]:
div = divs[0]
sub_href = div.select_one('.a-card__header > a')['href']
sub_href

'/questions/4533cd415d92bf728148a070ae17e366?status=waiting,ready,outdated&order=recent&aha_term=%EB%B6%80%EB%8F%99%EC%82%B0%20%ED%8C%81'

In [798]:
data = []
for div in divs:
    sub_href = div.select_one('.a-card__header > a')['href']
    sub_url = base_url + sub_href
    sub_res = requests.get(sub_url)
    sub_soup = BeautifulSoup(sub_res.text, 'html.parser')
    
    driver.execute_script("window.open('');")
    driver.switch_to.window(driver.window_handles[1])
    driver.get(sub_url)
    time.sleep(2)
    
    sub_soup2 = BeautifulSoup(driver.page_source, 'html.parser')
    qs = sub_soup2.select('.a-card.card-question > .a-card__main > .a-card__content > .editor__content > p')
    question = ''
    for q in qs:
        text = q.get_text().strip()
        if text == '\u200b' or text == '':
            continue
        question += text
    
    
    ans = sub_soup2.select('.a-card.cardAnswer > .a-card__main > .a-card__content > .editor__content > p')
    answer = ''
    for a in ans:
        text = a.get_text().strip()
        if text == '\u200b' or text == '':
            continue
        answer += text
        
    driver.close()
    time.sleep(2)
    driver.switch_to.window(driver.window_handles[-1])
    time.sleep(2)
    
    data.append({'구분': '팁', '질문': question, '답변': answer})

driver.close()
    

TypeError: 'NoneType' object is not subscriptable

In [803]:
import pandas as pd 
df = pd.DataFrame(data)
df

,구분,질문,답변
0,팁,아파트를 매도할때 집 앞에 부동산에만 얘기를 할지동네 부동산에 전부다 얘기를 할지 ...,안녕하세요. 한결같은고양이32입니다.최소3군데가봐야정확한가격이나와요부동산중계소마다조...
1,팁,제꿈이 부동산을 운영하면서 매매도 열심히해드리고 돈도 많이벌고싶은 22살입니다 부동...,안녕하세요. 박어상 공인중개사입니다.부동산을 운영하고 돈을 버는 것은 나이와는 전혀...
2,팁,사정이 좋지않아서 지금 매수세가 없지만 집을 팔아야하네요ㅜㅜ근처부동산에는 전화돌려보...,안녕하세요. 홍성택 공인중개사입니다.지금 부동산시장이 얼어붙었죠.다들 관망중인듯요....
3,팁,곧 계약이 다되가서 월세 방을 구해야하는데요이 집 구할때도 부동산 사람이 의도적으로...,안녕하세요. 서주환 공인중개사입니다.여러군데 공인중개사를 통해 다양한 매물을 확인 ...
4,팁,반려견이 2마리있는데 부동산을 하루에 15곳도 가봤지만하나같이 집이 없다 또는 집주...,안녕하세요. 양정섭 공인중개사입니다.안타깝지만 집주인이 반려견을 허락해야지 계약이 ...
...,...,...,...
305,팁,이번에 제가 부동산을 직거래 할려고 하는데 계약서는 어떻게 써야할 지 몰라서 부동산...,"안녕하세요. 김인혜 공인중개사입니다.공인중개사가 거래성과 없이 대필의뢰 받은 경우,..."
306,팁,일반음식점 운영중인데 계약기간 만료전 인수자를 구해 나가려고 합니다. 제가 직접 인...,"안녕하세요. 유창효 공인중개사입니다.1, 계약만료전 중도해지라면 중개보수에 대해서는..."
307,팁,안녕하세요. 수리수리마하수리12345입니다.최근에 부동산 경매에 관심이 있어 이것저...,안녕하세요. 김인혜 공인중개사입니다.경매에서 명도이전신청이라고 하면 명도소송 전에 ...
308,팁,아파트나 부동산같은것을 취득한 후에는 취등록세를 언제까지 납부를해야되는건가요??등기...,안녕하세요. 김인혜 공인중개사입니다.그 취득한 날부터 60일 이내에 그 과세표준에 ...


In [802]:
df.to_csv('data/팁309.csv')